# Executing Heterogeneous DAG Workflows with Parsl-RP (RPEX)
RPEX integrates the powerful runtime engine and workload manager of RADICAL-Pilot with the flexible and parallel workflow manager of Parsl. RPEX offers the best of both worlds by enabling users to run heterogeneous regular and MPI workflows, such as executables and Python functions, within the same environments on different HPC platforms. Users can express and manage these workflows via Parsl.


In this tutorial, we will explore the process of creating a Directed Acyclic Graph (DAG) comprising both MPI (Message Passing Interface) tasks and regular tasks using the Parsl API. The execution of the DAG's tasks will be orchestrated through the RPEX executor. This tutorial will demonstrate how to utilize Parsl's data flow manager and RADICAL Pilot's workload manager to achieve concurrent task execution within the DAG.

## Overview

The tutorial will cover the following key steps:

1. **Configuring the RPEX Executor**:
    - Setting up the RPEX executor and binding it to the DAG for task execution.


2. **Constructing a Heterogeneous DAG**:
    - Using Parsl API to define a heterogeneous DAG with both MPI and non-MPI tasks.


3. **Executing the DAG**:
    - Running the DAG utilizing RPEX on an local machine

<div style="text-align:center"><img src="https://airflow.apache.org/docs/apache-airflow/stable/_images/basic-dag.png" alt="my image" width="30%"></div>

As a best practice, let's ensure RADICAL-Pilot and Parsl exist in the notebook environment.

In [1]:
!pip show parsl && echo "==============" && ! radical-stack

Name: parsl
Version: 2024.1.22
Summary: Simple data dependent workflows in Python
Home-page: https://github.com/Parsl/parsl
Author: The Parsl Team
Author-email: parsl@googlegroups.com
License: Apache 2.0
Location: /home/aymen/ve/rpex/lib/python3.8/site-packages
Requires: globus-sdk, tblib, paramiko, dill, typeguard, typing-extensions, setproctitle, requests, psutil, six, pyzmq
Required-by: 

  python               : /home/aymen/ve/rpex/bin/python3
  pythonpath           : 
  version              : 3.8.10
  virtualenv           : /home/aymen/ve/rpex

  radical.gtod         : 1.46.0
  radical.pilot        : 1.46.1
  radical.saga         : 1.42.0-v1.41.0-1-g8da6a9c1@devel
  radical.utils        : 1.46.0



First, let's import Parsl and RP Python modules in our application, alongside the RadicalPilotExecutor (RPEX) from Parsl

In [2]:
import parsl
from parsl.config import Config
from parsl.app.app import python_app, bash_app
from parsl.executors.radical import ResourceConfig
from parsl.executors.radical import RadicalPilotExecutor

## Configuring the RPEX Executor

RPEX uses `ResourceConfig`, which is a data class that gives the flexibility to define advanced execution constraints for the RADICAL-Pilot runtime system, such as the number of workers and number of CPUs or GPUs per worker and more.

For the purpose of this tutorial, we will use `MPI` worker by specifying the `worker_type` parameter for the `ResourceConfig` class instance, which deploys one MPI worker with 4 CPU cores per worker and 0 GPUs.

In [3]:
rpex_cfg = ResourceConfig()
rpex_cfg.worker_type = 'MPI'
rpex_cfg.cores_per_worker = 4

<div class="alert alert-block alert-info">
⚠️ NOTE:
    
The ***cores*** on the executor level represent the entire amount of cores for the executor, including the MPI worker. This approach helps to create a clean separation between the number of cores that are used for the MPI workers, which are responsible for the function execution, and other resources that are used for running executable tasks, for example.
</div>

Once we create the `ResourceConfig`, we will pass it to the RPEX executor initialization. This will tell the executor to deploy 1 MPI worker with 4 cores and the rest of the 8 cores (4 cores) are left for executable tasks execution if any.

In [4]:
config = Config(executors=[RadicalPilotExecutor(
                           label='rpex-heterogeneous',
                           rpex_cfg=rpex_cfg,
                           resource='local.localhost',
                           runtime=30, cores=8)])

radical_executor = config.executors[0]

Now, let's tell Parsl that we want to use the RPEX executor and to do so we invoke the ``load`` function with the designated config of `RadicalPilotExecutor`.

In [5]:
parsl.load(config)

### Constructing a Heterogeneous DAG

Now the executor is started, let's construct the workflow with 1 MPI functions and 3 regular functions as follows:

- ``task_a``: MPI function that calacultes the mean in parallel for N lists 
- ``task_b``, ``task_c``: calaculate the square root of one of the means of ``task_a`` (``task_b`` and ``task_c`` will run concurrently)
- ``task_d``: calculate Euler's identity.

In [6]:
@python_app
def task_a(arrays, comm=None, parsl_resource_specification={}):
    """
    calculates the mean of an array, using MPI
    
    :param arr: (np.ndarray)
    :param comm: (MPI Communicators) if None, MPI.COMM_WORLD
    
    :return: (np.ndarray or Number) the result of the sum
    """
    import statistics

    rank = comm.Get_rank()
    size = comm.Get_size()

    if size != len(arrays):
        raise ValueError("Number of passed lists must be equal to number of ranks")

    if rank == 0:
        data = arrays[0]
        local_mean = statistics.mean(data)
    else:
        data = arrays[rank]
        local_mean = statistics.mean(data)

    global_means = comm.gather(local_mean, root=0)

    return global_means


arrays =  [[6, 7, 8, 9, 10], [16, 17, 18, 19, 20]]

mpi_means = task_a(arrays, comm=None,
                   parsl_resource_specification={'ranks': 2})

means = [m for m in mpi_means.result() if m is not None][0]

In [10]:
@python_app
def task_b_and_c(mean, comm=None, parsl_resource_specification={'ranks':1}):
    """
    calaculate the square root of the mean value
    
    :param mean: int value of mean
    
    :return: square root of mean
    """

    import math

    return math.sqrt(mean)

sqrt_b = task_b_and_c(means[0], comm=None)
sqrt_c = task_b_and_c(means[1], comm=None)

In [ ]:
sqrt_b.result()

In [ ]:
@python_app
def task_d(b, c, comm=None, parsl_resource_specification={'ranks':1}):
    import cmath

    # Compute Euler's identity
    result = cmath.exp(1j * cmath.pi) + 1

    return result

In [ ]:
print(task_d(sqrt_b, sqrt_c).result())

Finally, shutdown the executor, otherwise it will always stays ready to get more tasks

In [ ]:
radical_executor.shutdown()